# Retriever Custom e ParentDocumentRetriever

### Creazione Vector-Store

In [1]:
# lettura di tutti i documenti da una cartella

from langchain_community.document_loaders import DirectoryLoader  # pip install unstructured
                                                                  # https://pypi.org/project/python-magic-bin/0.4.14/#files scaricare e installare la wheel corretta

loader = DirectoryLoader('data/dialoghi', glob="**/*.txt") # carica tutto i file .txt dalla cartella e sottocartelle
documents = loader.load()

for doc in documents:
    print(doc, "\n\n")

page_content='Al negozio di abbigliamento Commesso - Buongiorno, posso aiutarla? Cliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli? Commesso - Certo, che taglia? Cliente - Porto una 50 Commesso - Eccoli qua! Cliente - Dove sono i camerini? Commesso - I camerini sono là in fondo a destra, accanto alle scale Cliente - Perfetto, grazie

Commesso - Come vanno? Cliente - Il modello mi piace, ma sono un po' stretti. Posso provare una taglia più larga? Commesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso modello in marrone? Cliente - No, grazie, il marrone proprio non mi piace. Non avete altri colori in questa taglia? Commesso - Allora, nella taglia 52 abbiamo il marrone, il rosso e il grigio. Cliente - Vabbè, li provo in grigio, vediamo come mi stanno.

Cliente - Ho provato i pantaloni, anche in grigio sono proprio belli e la taglia è perfetta! Quanto costano? Commesso - Costano 85€ Cliente - Ma non sono in sconto

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=documents, embedding=OpenAIEmbeddings())

In [3]:
for doc in vectorstore.similarity_search("mamma", k=1):
    print(doc.metadata["source"], doc.page_content[:50] + "...\n\n")

data/dialoghi/dialogo3.txt Davanti alla scuola Mamma 1 - Ciao ragazze! Come s...




In [4]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
res = retriever.invoke("mamme che parlano")
res

[Document(id='ad460a98-c140-4c17-9471-86dabdd7fc99', metadata={'source': 'data/dialoghi/dialogo3.txt'}, page_content="Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.\n\nMamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano. Mamma 1 - Pietro è bravo in tutte le materie, ma è molto disordinato. Trova sempre difficile tenere in ordine il suo zaino e i suoi quaderni.\n\nMamma 2 - Avete sentito de

# Retriever Custom
### tramite decoratore *chain*

Spesso è utile creare retriever custom per poter inserire ulteriori informazioni oltre a quelle direttamente fornite dal VectorStore.

In [5]:
query="mamme che parlano"
vectorstore.similarity_search_with_score(query)

[(Document(id='ad460a98-c140-4c17-9471-86dabdd7fc99', metadata={'source': 'data/dialoghi/dialogo3.txt'}, page_content="Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.\n\nMamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano. Mamma 1 - Pietro è bravo in tutte le materie, ma è molto disordinato. Trova sempre difficile tenere in ordine il suo zaino e i suoi quaderni.\n\nMamma 2 - Avete sentito d

In [6]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

# Grazie al decoratore @chain possiamo creare un runnable custom.
# Implementando un runnable che prende in input una query e restituisce una lista di Document
# a conti fatto abbiamo creato un retriever custom che, a questo punto, possiamo custimizzare come vogliamo.
#
# In questo caso, ci limitiamo a restituire i documenti con lo score di similarità che abbiamo ottenuto 
# tramite similarity_search_with_score (che restituisce una lista di tuple (Document, score)), inserito nei metadati
@chain
def retriever(query: str) -> List[Document]:
    results = vectorstore.similarity_search_with_score(query)
    for doc, score in results:
        doc.metadata["score"] = score
    return [doc for doc, _ in results]

res = retriever.invoke("mamme che parlano")

for r in res:
    print(r.page_content.replace("\n", "")[:25], "\t", r.metadata)

Davanti alla scuola Mamma 	 {'source': 'data/dialoghi/dialogo3.txt', 'score': 0.32034605741500854}
Al supermercato Cliente 1 	 {'source': 'data/dialoghi/dialogo4.txt', 'score': 0.444622665643692}
In ufficio, durante la pa 	 {'source': 'data/dialoghi/dialogo5.txt', 'score': 0.44506949186325073}
Al negozio di abbigliamen 	 {'source': 'data/dialoghi/dialogo.txt', 'score': 0.46766114234924316}


In [7]:
res

[Document(id='ad460a98-c140-4c17-9471-86dabdd7fc99', metadata={'source': 'data/dialoghi/dialogo3.txt', 'score': 0.32034605741500854}, page_content="Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.\n\nMamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano. Mamma 1 - Pietro è bravo in tutte le materie, ma è molto disordinato. Trova sempre difficile tenere in ordine il suo zaino e i suoi quaderni.

# ParentDocumentRetriever
### Recupero di un Chunk e del Documento che lo contiene 

Il ParentDocumentRetriever è un retriever che ci permette di utilizzare i chunk per le ricerche ma ci restituisce poi, non il chunk stesso, ma l'intero documento da cui quel chunk è stato estratto. In questo modo la ricerca è fatta su un vettore "piccolo" (il chunk) ma ci portiamo poi all'interno del prompt, non il chunk piccola, ma l'informazione completa che il modello può utilizzare per generare la risposta

In pratica, embeddizziamo e ricerchiamo chunk, ma poi restituiamo documenti interi

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400) # creiamo i chunk di 400 caratteri

vectorstore = Chroma(collection_name="full_documents", embedding_function=OpenAIEmbeddings())

store = InMemoryStore() # in questo modo lo storage avviene in memoria non persistente

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, 
    docstore=store, 
    child_splitter=child_splitter, 
)

# i documenti vengono chunkizzati e aggiunti al vectorstore
# mentre i documenti interi vengono aggiunti al docstore
retriever.add_documents(documents, ids=None)  

list(store.yield_keys())

['50448847-8296-483c-bc28-f3f84795098c',
 '183f0303-ae78-41bc-b156-e8861e8420c5',
 '065ec3a3-8be4-4ade-8a34-ff0a576eaf78',
 '431afd4f-fb3f-410e-8581-583b1f83037f',
 'aaf13234-fcb4-4dcd-83bd-73ea01f0e4b5']

In [9]:
# Nello store abbiamo solo i documenti interi
len(list(store.yield_keys()))

5

La query eseguita sul vector store, mi restituisce i chunk più simili

In [10]:
sub_docs = vectorstore.similarity_search("mamme che parlano")
print(sub_docs[0].page_content)
print("===")
print(len(sub_docs[0].page_content))

Mamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano.
===
394


Mentre, la stessa query eseguita sul retriever mi restituisce il documento intero


In [11]:
retrieved_docs = retriever.invoke("mamme che parlano")
print(retrieved_docs[0].page_content)
print("===")
print(len(retrieved_docs[0].page_content))

Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.

Mamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano. Mamma 1 - Pietro è bravo in tutte le materie, ma è molto disordinato. Trova sempre difficile tenere in ordine il suo zaino e i suoi quaderni.

Mamma 2 - Avete sentito della nuova maestra di inglese? Dicono che sia molto brava. Mamma 3 - Sì, mia figlia ne è entusiasta! Finalmente qualcuno ch

## Gestione di Chunk multi-livello

In alcuni casi però, recuperare l'intero documento da cui il chunk è stato creato, rischia di farci recurare documenti eccessivamente grandi che non potrebbero poi essere utilizzati all'interno del prompt.

Per gestire questo problema possiamo usare dei chunk multi-livello. L'idea di base è quella di *recuperare piccole parti di un chunk, per poi recuperare il chunk da cui sono state generate.*    

In [28]:
# Text-Splitter per i chunk di documento più grandi, da usare come dati recuperati
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=150)

# Secondo Text-Splitter, per la creazione di chunk più piccoli su cui inidicizzare le ricerche
child_splitter = RecursiveCharacterTextSplitter(chunk_size=85, chunk_overlap=15)

vectorstore = Chroma(collection_name="split_parents", embedding_function=OpenAIEmbeddings())

# dizionario in memoria utilizzato per recuperare i documenti più grandi, una volta
# identificati tramite ricerca sui chunk più piccoli
store = InMemoryStore()

In [29]:
# ParentDocumentRetriever gestisce automaticamente
# il "doppio livello" di granularità dei chunk

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# i documenti vengono splittati in chunk più grandi (parent) e poi in chunk più piccoli (child)
# i chunk più piccoli vengono indicizzati nel vectorstore, mentre i chunk più grandi
# vengono memorizzati nel docstore
retriever.add_documents(documents)

# Rispetto a prima, adesso nello store abbiamo molti più documenti
# perché prima avevamo solo i documenti interi, mentre adesso
# abbiamo i chunk più grandi (parent). Ovviamente nel vectorstore
# abbiamo molti più chunk
len(list(store.yield_keys()))

57

In [18]:
# accedendo al VectrorStore (Chroma), tramite similarity search, la ricerca viene effettuata
# sui chunk estratti più piccoli
sub_docs = vectorstore.similarity_search("mamme che parlano")

print(sub_docs[0].page_content)
print("===")
print(len(sub_docs[0].page_content))

i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1
===
83


In [15]:
# il retriever ritorna invece il chunk di documento più grande
retrieved_docs = retriever.invoke("mamme che parlano")

print(retrieved_docs[0].page_content)
print("===")
print(len(retrieved_docs[0].page_content))

Mamma 2 - Cambiando argomento, cosa farete questo fine settimana? Mamma 3 - Noi andiamo al parco, sperando che il tempo sia bello. I bambini hanno bisogno di correre un po'. Mamma 4 - Noi invece visiteremo i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1 - Noi pensavamo di andare al cinema. C'è un nuovo film per
===
348


In [16]:
for r in retrieved_docs:
    print(r)

page_content='Mamma 2 - Cambiando argomento, cosa farete questo fine settimana? Mamma 3 - Noi andiamo al parco, sperando che il tempo sia bello. I bambini hanno bisogno di correre un po'. Mamma 4 - Noi invece visiteremo i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1 - Noi pensavamo di andare al cinema. C'è un nuovo film per' metadata={'source': 'data/dialoghi/dialogo3.txt'}
page_content='i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1 - Noi pensavamo di andare al cinema. C'è un nuovo film per bambini che sembra carino. Mamma 2 - Buone idee! Anche noi potremmo fare un'escursione in montagna. I bambini adorano esplorare.' metadata={'source': 'data/dialoghi/dialogo3.txt'}
page_content='Mamma 2 - Avete sentito della nuova maestra di inglese? Dicono che sia molto brava. Mamma 3 - Sì, mia figlia ne è entusiasta! Finalmente qualcuno che rende le lezioni interessanti. Mamma 4 - Speriamo bene. Emma ha sempre trovato diffic